<a href="https://colab.research.google.com/github/gaurishankar9793/ML-Hikeathon/blob/master/ML_Hikethon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



    100% |████████████████████████████████| 993kB 19.9MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
import os
file_list = drive.ListFile().GetList()


for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname =  f['title']
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  

title: train.zip, id: 1LLyOF4zWAXIdsBWGlYnD2exH5IypajOX
downloading to train.zip
title: test.zip, id: 1F0powCtTQxOdGYdgFhEcNUFRLIyTRkLt
downloading to test.zip
title: Copy of test.zip, id: 1YORMNqPeKvIymNU1ukhL_QoydeNQfyww
downloading to Copy of test.zip
title: hikeathon, id: 1Rv9m-nyQ1fmJ1shIwwVttvMAxKQN8o4K
downloading to hikeathon


FileNotDownloadableError: ignored

In [0]:
fid = drive.ListFile().GetList()[0]['id']

In [0]:
import zipfile
import pandas as pd
 
zf = zipfile.ZipFile('train.zip') 
train_df = pd.read_csv(zf.open('train.csv'))
train_features = pd.read_csv(zf.open('user_features.csv'))


In [0]:
display(train_df.shape)
display(train_df.is_chat.value_counts())

(70661802, 3)

0    68410018
1     2251784
Name: is_chat, dtype: int64

In [0]:
t2 = train_df.iloc[:100000,:]
train_features = train_features.set_index('node_id')

In [0]:
cols = train_features.columns
for col in cols:
  t2['1'+col] = t2['node1_id'].apply(lambda x: train_features.loc[x,col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
for col in cols:
  t2['2'+col] = t2['node2_id'].apply(lambda x: train_features.loc[x,col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
t2.head()

,node1_id,node2_id,is_chat,1f1,1f2,1f3,1f4,1f5,1f6,1f7,...,2f4,2f5,2f6,2f7,2f8,2f9,2f10,2f11,2f12,2f13
0,8446602,6636127,0,22,0,0,24,0,0,24,...,15,11,5,19,12,5,16,10,4,10
1,1430102,7433949,0,0,0,0,0,0,0,0,...,16,13,6,20,16,7,25,21,6,15
2,2803017,8372333,0,9,2,0,14,7,1,14,...,1,1,0,1,1,0,0,0,0,8
3,4529348,894645,0,11,11,10,11,10,9,10,...,1,1,1,1,1,1,1,1,1,15
4,5096572,4211638,0,31,31,31,31,31,31,31,...,7,1,0,9,1,0,16,1,0,7


In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
X = t2.iloc[:,3:].values.astype(float)
Y = t2.loc[:,'is_chat'].values


In [0]:
X.shape
seed = 7

In [0]:
model = Sequential()
model.add(Dense(60, input_dim=26, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, input_dim=60, kernel_initializer='normal', activation='relu'))
model.add(Dense(20, input_dim=100, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.fit(X,Y, epochs = 10, batch_size = 1000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100000/100000 [==============================] - 2s 17us/step - loss: 0.2064 - acc: 0.9667
Epoch 2/10
100000/100000 [==============================] - 1s 8us/step - loss: 0.1250 - acc: 0.9677
Epoch 3/10
100000/100000 [==============================] - 1s 8us/step - loss: 0.1236 - acc: 0.9677
Epoch 4/10
100000/100000 [==============================] - 1s 8us/step - loss: 0.1225 - acc: 0.9677
Epoch 5/10
100000/100000 [==============================] - 1s 8us/step - loss: 0.1220 - acc: 0.9677
Epoch 6/10
100000/100000 [==============================] - 1s 7us/step - loss: 0.1211 - acc: 0.9677
Epoch 7/10
100000/100000 [==============================] - 1s 7us/step - loss: 0.1210 - acc: 0.9676
Epoch 8/10
100000/100000 [==============================] - 1s 8us/step - loss: 0.1199 - acc: 0.9676
Epoch 9/10
100000/100000 [==============================] - 1s 7us/step - loss: 0.1200 - acc: 0.9676
Epoch 10/10
100000/100000 [===============

In [0]:
model.predict(X)

array([[0.01735255],
       [0.03856307],
       [0.01228482],
       ...,
       [0.2729227 ],
       [0.02515772],
       [0.0030973 ]], dtype=float32)

In [0]:
import zipfile
import pandas as pd
 
zf2 = zipfile.ZipFile('./test.zip') 
test_df = pd.read_csv(zf2.open('test.csv'))
# train_features = pd.read_csv(zf.open('user_features.csv'))

In [0]:
test_df.shape

(11776968, 3)

In [0]:

l =[]
t =0
for i in range(59):
  print(i)
  test1 = test_df.iloc[t:t+200000,:]


  for col in cols:
    test1['1'+col] = test1['node1_id'].apply(lambda x: train_features.loc[x,col])

  for col in cols:
    test1['2'+col] = test1['node2_id'].apply(lambda x: train_features.loc[x,col])

  l.append(model.predict(test1.iloc[:,3:].values))
  t+=200000
  

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


In [0]:
len(l)


59

In [0]:
59*200000

11800000

In [0]:
test_df.shape

(11776968, 3)

In [0]:
flat_list = [item for sublist in l for item in sublist]

In [0]:
len(flat_list)

11776968

In [0]:
test_df['Score'] = flat_list
test_df.head()

,id,node1_id,node2_id,Score
0,1,7107094,8010772,[0.013597041]
1,2,7995251,2805801,[0.027720392]
2,3,2804693,8059549,[0.0037758946]
3,4,4812472,7332370,[0.0013335943]
4,5,5009985,4511909,[0.025363326]


In [0]:
test_f = test_df[['id','Score']]
test_f.shape

(11776968, 2)

In [0]:
# test_f['Score'] = test_f['Score'].values
type(test_f.Score[0])

numpy.ndarray

In [0]:
test_f['is_chat'] = test_f['Score'].apply(lambda x: x[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
test_f.drop(columns = 'Score', inplace = True)
test_f.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,is_chat
0,1,0.013597
1,2,0.027720
2,3,0.003776
3,4,0.001334
4,5,0.025363


In [0]:
test_f[['id','is_chat']].to_csv('prediction.csv', index= False)

In [0]:
from google.colab import files
files.download('prediction.csv') 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 35020, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil